In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression

import fl_utils.fl_interpretable_functions as interpret
import fl_utils.fl_riskslim as slim
import fl_utils.fl_stumps as stumps

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\ky-fl-combined\fl-model


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### EBM & CART

In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/KY-cleaned.csv").sort_values('person_id')
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/FL-cleaned.csv").sort_values('person_id')

## split x and y 
KY_X = KY_data.loc[:,:'current_violence20']
KY_Y = KY_data['general_two_year'].values
FL_X = FL_data.loc[:,:'current_violence20']
FL_Y = FL_data['general_two_year'].values

In [4]:
#### CART
depth = [1,2,3,4,5]
impurity = [0.001, 0.002, 0.003, 0.004, 0.005]
cart_summary = interpret.CART(KY_X, KY_Y, FL_X, FL_Y, depth, impurity, seed=816)

### EBM
estimators = [40,60,80,100]
depth = [1,2,3]
learning_rate = [0.01]
holdout_split = [0.7, 0.9]
ebm_summary = interpret.EBM(KY_X, KY_Y, FL_X, FL_Y, learning_rate, depth, estimators, holdout_split, seed=816)

In [5]:
print("CART: ", np.mean(cart_summary['KY_score']))
print("EMB: ", np.mean(ebm_summary['KY_score']))

CART:  0.6256025696786767
EMB:  0.7041386731681298


---

### Lasso Stumps

In [6]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/KY-stumps.csv").sort_values('person_id')
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/FL-stumps.csv").sort_values('person_id')

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201'].copy()
KY_Y = KY_stumps['general_two_year'].values.copy()
FL_X = FL_stumps.loc[:,:'current_violence201'].copy()
FL_Y = FL_stumps['general_two_year'].values.copy()

## columns 
cols = KY_X.columns[1:]

In [8]:
single_stump_model = stumps.stump_model(KY_X, 
                                        KY_Y, 
                                        FL_X, 
                                        FL_Y, 
                                        0.08, 
                                        cols, 
                                        816)
## unique original features
unique_stumps = []
for i in single_stump_model['features']:
    unique_stumps.append(''.join([j for j in i if not j.isdigit()]))
print(len(np.unique(unique_stumps)))

14


In [10]:
c_grid={'C': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08]}
stumps_summary = stumps.stump_cv(KY_X, KY_Y, FL_X, FL_Y, cols, c_grid, seed=816)

In [11]:
print("CART: ", np.mean(cart_summary['KY_score']))
print("EMB: ", np.mean(ebm_summary['KY_score']))
print("Additive: ", np.mean(stumps_summary['KY_score']))

CART:  0.6256025696786767
EMB:  0.7041386731681298
Additive:  0.652888833143239


---

### RiskSLIM

In [12]:
single_stump_model = stumps.stump_model(KY_X, 
                                        KY_Y,
                                        FL_X,
                                        FL_Y,
                                        0.07,
                                        cols, 816)
selected_features = single_stump_model['features']
len(selected_features)

21

In [13]:
### Subset features
sub_FL_X = FL_stumps.loc[:, selected_features]
sub_KY_X = KY_stumps.loc[:, selected_features]
sub_FL_X.insert(0, '(Intercept)', 1)

In [14]:
riskslim_summary = slim.risk_cv(sub_KY_X, 
                                KY_Y, 
                                sub_FL_X, 
                                FL_Y, 
                                y_label ='general_two_year', 
                                max_coef = 5, 
                                max_coef_number = 5, 
                                max_runtime=1000,
                                max_offset=100,
                                c=1e-6, 
                                seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
04/04/20 @ 03:09 PM | 263 rows in lookup table
04/04/20 @ 03:09 PM | ------------------------------------------------------------
04/04/20 @ 03:09 PM | runnning initialization procedure
04/04/20 @ 03:09 PM | ------------------------------------------------------------
04/04/20 @ 03:09 PM | CPA produced 2 cuts
04/04/20 @ 03:09 PM | running naive rounding on 8 solutions
04/04/20 @ 03:09 PM | best objective value: 0.7125
04/04/20 @ 03:09 PM | rounding produced 5 integer solutions
04/04/20 @ 03:09 PM | best objective value is 0.7002
04/04/20 @ 03:09 PM | running sequential rounding on 8 solutions
04/04/20 @ 03:09 PM | best objective value: 0.7125
04/04/20 @ 03:09 PM | sequential rounding produced 0 integer solutions
04/04/20 @ 03:09 PM | polishing 5 solutions
04/04/20 @ 03:09 PM | best objective value: 0.7002
04/04/20 @ 03:09 PM | polishing produced 5 integer solutions
04/04/20 @ 03:09 PM | initialization produced 8 feasible soluti

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
04/04/20 @ 03:09 PM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.6727.
Tried aggregator 1 time.
Reduced MIP has 23 rows, 46 columns, and 87 nonzeros.
Reduced MIP has 21 binaries, 23 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

04/04/20 @ 03:09 PM | best objective value: 0.6688
04/04/20 @ 03:09 PM | ------------------------------------------------------------
04/04/20 @ 03:09 PM | completed initialization procedure
04/04/20 @ 03:09 PM | ------------------------------------------------------------
04/04/20 @ 03:09 PM | 263 rows in lookup table
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_Parallel                                1
CPXPARAM_RandomSeed                              0
CPXPARAM_TimeLimit                               1000
CPXPARAM_MIP_Tolerances_LowerCutoff              0.634974191787679
CPXPARAM_MIP_Tolerances_UpperCutoff              0.66884130025520405


Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
04/04/20 @ 03:09 PM | adding 214 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.6688.
Tried aggregator 1 time.
Reduced MIP has 23 rows, 46 columns, and 87 nonzeros.
Reduced MIP has 21 binaries, 23 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

 135351 16732        0.6504     8        0.6568        0.6458   633065    1.69%           rho_4 U 135351 106141     25
 138321 15973        0.6463     7        0.6568        0.6463   645152    1.60%           rho_4 D 138321  62120     27
 141367 15029        0.6505     1        0.6568        0.6469   657029    1.52%           rho_4 U 141367  63920     25
 144253 13859        cutoff              0.6568        0.6476   668872    1.40%           rho_0 U 144253 144252     24
 147137 12879        cutoff              0.6568        0.6481   679607    1.33%           rho_2 U 147137  74813     32
 150115 11790        cutoff              0.6568        0.6486   690874    1.25%          rho_18 U 150115 150114     28
 152917 10551        cutoff              0.6568        0.6494   701722    1.14%           rho_4 U 152917 113051     27
 155736  9315        cutoff              0.6568        0.6500   712760    1.04%           rho_4 U 155736 155734     18
 158683  8311        0.6547     5        0.6568 

...

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
04/04/20 @ 03:26 PM | adding 219 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.6696.
Tried aggregator 1 time.
Reduced MIP has 23 rows, 46 columns, and 87 nonzeros.
Reduced MIP has 21 binaries, 23 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Elapsed time = 75.77 sec. (41258.67 ticks, tree = 2.40 MB, solutions = 6)
 141223  2229        cutoff              0.6628        0.6604   576026    0.36%           rho_7 U 141223  48136     37

Gomory fractional cuts applied:  1
User cuts applied:  522

Root node processing (before b&c):
  Real time             =    0.09 sec. (1.98 ticks)
Sequential b&c:
  Real time             =   77.84 sec. (43161.27 ticks)
                          ------------
Total (root+branch&cut) =   77.94 sec. (43163.25 ticks)
setting c0 = 0.0 to ensure that intercept is not penalized
04/04/20 @ 03:27 PM | 263 rows in lookup table
04/04/20 @ 03:27 PM | ------------------------------------------------------------
04/04/20 @ 03:27 PM | runnning initialization procedure
04/04/20 @ 03:27 PM | ------------------------------------------------------------
04/04/20 @ 03:27 PM | CPA produced 2 cuts
04/04/20 @ 03:27 PM | running naive rounding on 5 solutions
04/04/20 @ 03:27 PM | best objective value: 0.6952
04/04/20 @ 

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
04/04/20 @ 03:27 PM | adding 223 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.6931.
Tried aggregator 1 time.
Reduced MIP has 23 rows, 46 columns, and 87 nonzeros.
Reduced MIP has 21 binaries, 23 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

04/04/20 @ 03:28 PM | best objective value: 0.7139
04/04/20 @ 03:28 PM | rounding produced 5 integer solutions
04/04/20 @ 03:28 PM | best objective value is 0.6879
04/04/20 @ 03:28 PM | running sequential rounding on 8 solutions
04/04/20 @ 03:28 PM | best objective value: 0.7139
04/04/20 @ 03:28 PM | sequential rounding produced 0 integer solutions
04/04/20 @ 03:28 PM | polishing 5 solutions
04/04/20 @ 03:28 PM | best objective value: 0.6879
04/04/20 @ 03:28 PM | polishing produced 5 integer solutions
04/04/20 @ 03:28 PM | initialization produced 9 feasible solutions
04/04/20 @ 03:28 PM | best objective value: 0.6814
04/04/20 @ 03:28 PM | ------------------------------------------------------------
04/04/20 @ 03:28 PM | completed initialization procedure
04/04/20 @ 03:28 PM | ------------------------------------------------------------
04/04/20 @ 03:28 PM | 263 rows in lookup table
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPX

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
04/04/20 @ 03:28 PM | adding 237 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.6814.
Tried aggregator 1 time.
Reduced MIP has 23 rows, 46 columns, and 87 nonzeros.
Reduced MIP has 21 binaries, 23 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

In [15]:
print("CART: ", np.mean(cart_summary['KY_score']))
print("EMB: ", np.mean(ebm_summary['KY_score']))
print("Additive: ", np.mean(stumps_summary['KY_score']))
print("Additive: ", np.mean(riskslim_summary['KY_score']))

CART:  0.6256025696786767
EMB:  0.7041386731681298
Additive:  0.652888833143239
Additive:  0.6494990099885669


In [16]:
#### save results
summary_general2_fl_inter_model = {"cart": cart_summary,
                                   "ebm": ebm_summary,
                                   "stumps": stumps_summary,
                                   "riskslim": riskslim_summary}

In [17]:
path = "./results/interpretable/two-year/"
results = [["", "CART", "EBM", "Lasso Stumps", "RiskSLIM"],
           ["General",             
            np.str(round(np.mean(cart_summary['KY_score']),3)) + " (" + np.str(round(np.std(cart_summary['KY_score']),3)) + ")", 
            np.str(round(np.mean(ebm_summary['KY_score']), 3)) + " (" + np.str(round(np.std(ebm_summary['KY_score']),3)) + ")", 
            np.str(round(np.mean(stumps_summary['KY_score']),3)) + " (" + np.str(round(np.std(stumps_summary['KY_score']),3)) + ")", 
            np.str(round(np.mean(riskslim_summary['KY_score']), 3)) + " (" + np.str(round(np.std(riskslim_summary['KY_score']),3)) + ")"]]
with open(path + 'two-year-fl-interpretable.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)